In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [66]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [67]:
covid_df = pd.read_excel('แบบสอบถามเรื่องผลกระทบจาก COVID-19 กับนักพัฒนาซอฟต์แวร์_shortened (Responses)-2nd_round.xlsx')
original_covid_df = covid_df.copy()

In [68]:
# covid_df.head()


In [69]:
covid_df.shape

(167, 163)

In [70]:
covid_df.columns.tolist()

['Timestamp',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมากที่สุด',
 'D1 ท่า

In [71]:
covid_df.describe()

,I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?,I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?,I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?
count,146.000000,145.000000,146.000000
mean,7.301370,7.751724,7.698630
std,1.371482,1.402037,1.599008
min,1.000000,1.000000,2.000000
25%,6.250000,8.000000,7.000000
50%,8.000000,8.000000,8.000000
75%,8.000000,8.000000,9.000000
max,10.000000,10.000000,10.000000


In [72]:

covid_df = covid_df.rename(columns={'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว':'Confirm_REMOVE',
'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน':'Office_work_before_REMOVE',
'A1 ตำแหน่งงานปัจจุบันของท่าน':'RolesIncludeDeveloper',
'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ':'YearsOfExperience',
'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?':'YearsOfWorkFromHomeExperience',
'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ':'Fulltime',
'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?':'Coinhabitants',
'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?':'Children',
'A7 ท่านอายุเท่าไหร่?':'Age',
'A8 โปรดระบุเพศของท่าน':'Gender',
'A9 ท่านจบการศึกษาสูงสุดในระดับใด?':'Education',
'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?':'OrganizationSize',
'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?':'Disabilities',
'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด':'B1_REMOVE',
'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน':'COVIDStatus',
'C2 เลือกข้อที่ตรงกับท่านมากที่สุด':'Isolation',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะติดเชื้อ COVID-19]':'FR1',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR2',
'D1 ท่านรู้สึกกลัวว่า [คนในครอบครัวของท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR3',
'D2 ตอนนี้ท่านรู้สึก [หวาดกลัว]':'FR4',
'D2 ตอนนี้ท่านรู้สึก [ตนเองไม่สามารถทำอะไรเพื่อแก้ปัญหาได้]':'FR5',
'D2 ตอนนี้ท่านรู้สึก [ตกใจกลัวอย่างที่สุด]':'FR6',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านหวาดกลัวว่าท่านจะไม่มีเงินหากท่านไม่สามารถทำงานได้ในช่วง 2-3 เดือนข้างหน้าหรือไม่?]':'FR7',
'D3 โปรดตามคำถามต่อไปนี้ [ปกติแล้วท่านสามารถหายจากอาการเจ็บป่วยได้รวดเร็วหรือไม่?]':'FR8',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านมีเพื่อนที่สามารถติดต่อได้ทางโทรศัพท์ อีเมล หรืออื่นๆ หรือไม่ ในเวลาที่ท่านรู้สึกโดดเดี่ยว]':'FR9',
'How many different prescription medications are you on?':'FR10',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปรึกษากับคนอื่นเกี่ยวกับข้อมูลเรื่อง COVID-19 ที่ข้าพเจ้าได้รับมา]':'DP1',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ามีชุดยังชีพฉุกเฉิน]':'DP2',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าได้วางแผนไว้แล้วว่าต้องปฏิบัติตัวอย่างไรหากข้าพเจ้าตรวจพบเชื้อ COVID-19]':'DP3',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปฏิบัติตามคำแนะนำของหน่วยงานของรัฐเกี่ยวกับการปฏิบัติตัวช่วง COVID-19 อย่างเคร่งครัด]':'DP4',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ารู้จักคนที่จะช่วยข้าพเจ้าได้หากข้าพเจ้าจะต้องกักตัว]':'DP5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [สถาพแวดล้อมของที่ทำงานที่บ้านของข้าพเจ้าปราศจากสิ่งรบกวน]':'Erg1',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ระดับของเสียงรบกวนที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่ยอมรับได้ (ไม่รบกวนการทำงาน)]':'Erg2',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [อุณหภูมิของที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg3',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [แสงในที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg4',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [เก้าอี้ทำงานของข้าพเจ้ามีความสบาย]':'Erg5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ที่ทำงานที่บ้านของข้าพเจ้ามีความเหมาะสมกับการทำงาน]':'Erg6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ข้าพเจ้าสามารถนำอุปกรณ์ (เช่น จอภาพ) จากที่ทำงานกลับมาที่บ้านได้]':'OS1',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้ออุปกรณ์ใหม่ให้พนักงานเพื่อทำงานที่บ้าน]':'OS2',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้อซอฟต์แวร์ที่จำเป็นสำหรับการทำงานที่บ้าน]':'OS3',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะจ่ายค่าอินเตอร์เน็ตให้ข้าพเจ้า]':'OS4',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะไม่ถูกไล่ออก]':'OS5',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะได้รับเงินเดือนเหมือนเดิม]':'OS6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าสามารถลาหยุดหากข้าพเจ้าป่วยหรือลาเพื่อไปดูแลคนในครอบครัวได้]':'OS7',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าองค์กรมีความเข้าใจหากผลการทำงานของข้าพเจ้าแย่ลง]':'OS8',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ายังคงมีการประชุมประจำเช่นเดิม (เช่น ผ่านระบบ video call)]':'OS9',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการประชุม (เช่น ผ่าน video call) น้อยลง]':'OS10',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าให้กำลังใจพนักงาน โดยให้มาพบเจอกัน (แบบออนไลน์) เป็นประจำ]':'OS11',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการสร้างกิจกรรม virtual social events (ผ่านระบบ via video chat) ให้ทุกคนในทีม]':'OS12',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ในแต่ละวัน ข้าพเจ้าทำงานโดยเปิด video call หรือ audio call ค้างไว้กับเพื่อนร่วมงานในทีม]':'OS13',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าส่งอาหารให้กับพนักงานที่บ้าน]':'OS14',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้บุตรของข้าพเจ้าได้ทำระหว่างที่ข้าพเจ้าทำงาน]':'OS15',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้พนักงานออกกำลังกายที่บ้าน]':'OS16',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าสนับสนุนให้พนักงานใช้เวลาช่วงนี้ในการฝึกฝนตนเอง (professional training)]':'OS17',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีระบบที่รองรับการทำงานที่บ้านได้อย่างดี (เช่น source control, VPN, remote desktop, file sharing)]':'OS18',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการตรวจเช็ค commits, change request หรือ pull request (peer code review)]':'OS19',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้าใช้ระบบ build system เพื่อทำการ compile และ test software]':'OS20',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ากำลังปรับปรุงเอกสารและกระบวนการทำงาน (เช่น การ approve change request/pull request)]':'OS21',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [มีคนในองค์กรที่ช่วยจัดลำดับความสำคัญของงานและตรวจเช็คข้อมูลใน backlog ให้ถูกต้องเสมอ]':'OS22',
'F2 มีสิ่งอื่นใดที่บริษัทของท่านกำลังทำหรือควรจะทำเพื่อช่วยท่านระหว่างทำงานที่บ้าน?':'Temp1',
'F3 มีสิ่งใดที่บริษัทของท่านทำแล้วท่านคิดว่าทำให้การทำงานที่บ้านลำบากมากขึ้น?':'Temp2',

'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5B1',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5B2',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5B3',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5B4',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนใจ]':'WHO5B5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB1',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB2',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านเลือกที่จะไม่ทำงานทั้งที่ควรจะต้องทำงานนั้นๆ บ่อยแค่ไหน?]':'HPQB3',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQB4',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQB5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQB6',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQB7',
'G3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQB8',
'G4 ตอนที่ท่านทำงานที่ออฟฟิศ ในบริเวณโต๊ะที่ท่านทำงานมีคนจำนวนกี่คน (รวมตัวท่านเอง)?':'Temp3',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านมีประชุมวันละประมาณกี่ครั้ง?':'Temp4',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',
'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',
'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5S1',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5S2',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5S3',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5S4',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนจ]':'WHO5S5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS1',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS2',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่ทำงานทั้งที่ควรจะต้องทำงานบ่อยแค่ไหน?]':'HPQS3',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQS4',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQS5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQS6',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQS7',
'H3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ตั้งแต่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQS8',
'H4 ตอนที่ท่านทำงานที่บ้าน ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp7.1',
'H5 ท่านทำงานที่บ้านเป็นประจำหรือไม่ก่อนเกิดการแพร่ระบาดของ COVID-19?':'Temp8',
'H5 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',
'H6 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',
#'H8 โปรดอธิบายว่าสถานที่ทำงานที่บ้านของท่านเป็นอย่างไร ':'Temp11',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Laptop]':'Temp12',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Desktop]':'Temp13',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Multiple screens]':'Temp14',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Headphones]':'Temp15',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Keyboard]':'Temp16',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Mouse]':'Temp17',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะ]':'Temp18',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Whiteboard]':'Temp19',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะแบบ Ergonomic]':'Temp20',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [เก้าอี้แบบ Ergonomic]':'Temp21',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [แสงสว่างตามธรรมชาติ]':'Temp22',
'H7 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านดีกว่าที่ทำงาน]':'Temp24',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ดี]':'Temp25',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [บริหารจัดการงานได้ดีขึ้น (เช่น การจัดลำดับความสำคัญของงาน การเลือกงาน)]':'Temp26',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาที่โฟกัสกับงานมากขึ้น]':'Temp27',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีการประชุมที่มีประสิทธิภาพขึ้น]':'Temp28',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีส่ิงรบกวนน้อยลง]':'Temp29',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใช้เวลาเดินทางน้อยลง]':'Temp30',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานที่ยืดหยุ่นขึ้น]':'Temp31',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีกิจกรรมออกกำลังกายมากขึ้น]':'Temp32',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้หยุดพักระหว่างทำงานบ่อยขึ้น]':'Temp33',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีปัญหาสุขภาพน้อยลง]':'Temp34',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใด้อยู่ใกล้ชิดกับครอบครัวมากขึ้น]':'Temp35',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีค่าใช้จ่ายน้อยลงในการเดินทาง อาหาร อื่นๆ]':'Temp36',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้สวมใส่เสื้อผ้าที่สบายมากขึ้น]':'Temp37',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานให้เสร็จมากขึ้น]':'Temp38',
'H9 มีประโยชน์อื่นๆ ที่ท่านได้รับจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp39',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านแย่กว่าที่ทำงาน]':'Temp40',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีอุปกรณ์ hardware, monitor หรืออุปกรณ์ที่จำเป็นไม่เพียงพอ]':'Temp41',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ไม่ดี]':'Temp42',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีส่ิงรบกวนมากขึ้น]':'Temp43',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีปัญหาการเชื่อมต่อ (เช่น Internet, VPN, Remote desktop)]':'Temp44',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [รับรู้ว่าเพื่อนร่วมงานกำลังทำอะไรอยู่น้อยลง]':'Temp45',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สื่อสารกับเพื่อนร่วมงานยาก]':'Temp46',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความยากลำบากในการใช้งานเครื่องมือที่ช่วยทำงานร่วมกัน (collaboration tools)]':'Temp47',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่สามารถทำงานได้ เพราะต้องคอยงานจากเพื่อนร่วมงาน]':'Temp48',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีประชุมเยอะเกินไป]':'Temp49',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีกิจกรรมออกกำลังกายน้อยลง]':'Temp50',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดการปฏิสัมพันธ์กับเพื่อนร่วมงาน]':'Temp51',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ได้หยุดพักระหว่างทำงานน้อยลง]':'Temp52',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ทำกิจกรรมประจำได้น้อยลง]':'Temp53',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดเส้นแบ่งระหว่างงานและชีวิตส่วนตัว]':'Temp54',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีเวลาในการทำงานให้เสร็จน้อยลง]':'Temp55',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีคนดูแลบุตร]':'Temp56',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีกำลังใจในการทำงาน]':'Temp57',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความลำบากในการหาอาหาร]':'Temp58',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ที่ทำงานที่บ้านมี ergonomic ต่ำ]':'Temp59',
'H11 มีปัญหาหรือความลำบากอื่นๆ ที่ท่านได้ประสบจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp60',
#'H15 เครื่องมือซอฟต์แวร์ที่ช่วยในการทำงาน (software tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp61',
#'H16 เครื่องมือฮาร์ดแวร์ที่ช่วยในการทำงาน (hardware tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp62',
#'H17 กระบวนการทำงาน (process) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp63',
'H12 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อดีกับท่านอย่างไร?':'Temp64',
'H13 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อเสียกับท่านอย่างไร?':'Temp65',
'I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?':'PerformanceOthers',
'I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?':'PerB1item',
'I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?':'PerS1item',
'I4 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับประสิทธิผลในการทำงานหรือไม่':'Temp65.1',
'J1 เปรียบเทียบกับการทำงานในออฟฟิศ ประสิทธิผล (productivity) ของทีมท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp66',
'J2 วัฒนธรรมการทำงานของทีมของท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp67',
'J3 ทีมของท่านช่วยเหลือท่านอย่างไรบ้างในช่วงการแพร่ระบาดของ COVID-19?':'Temp68',
'J4 เมื่อเปรียบเทียบกับการทำงานในออฟฟิศ ท่านคิดว่าความสามารถของทีมในการทำงานเพื่อให้เสร็จสิ้นให้ทันเดดไลน์เปลี่ยนแปลงไปหรือไม่? หากใช่ โปรดอธิบาย':'Temp69',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการได้รับข้อมูลใดๆ จากเพื่อนร่วมทีม]':'Temp70',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การรับรู้ว่าเพื่อนร่วมทีมกำลังทำงานอะไรอยู่]':'Temp71',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ประสิทธิภาพของการสื่อสารกับเพื่อนร่วมทีม]':'Temp72',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความง่ายในการสื่อสารกับเพื่อนร่วมทีม]':'Temp73',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [งานที่ข้าพเจ้าทำให้กับทีม]':'Temp74',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับเพื่อนร่วมทีม]':'Temp75',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับหัวหน้างาน]':'Temp76',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบประจำ]':'Temp77',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบไม่ได้นัดมาก่อน]':'Temp78',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [คุณภาพของการประชุมแบบประจำ]':'Temp79',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความรู้สึกของการได้มีปฏิสัมพันธ์ทางสังคมกับทีม]':'Temp80',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การแบ่งปันความรู้ในทีม]':'Temp81',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการทานอาหาร หรือ coffee break ร่วมกันแบบ virtual]':'Temp82',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการพบปะเพื่อพูดคุยกันแบบ 1-1]':'Temp83',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [แบ่งปันภาพถ่ายขำขันหรือเรื่องตลก]':'Temp84',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เล่นเกมส์ออนไลน์ด้วยกัน]':'Temp85',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการตั้ง online meeting channel เพื่อกิจกรรมสนุกสนาน]':'Temp86',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เริ่มการประชุมหรือจบการประชุมด้วยเรื่องที่ไม่เกี่ยวกับงาน]':'Temp87',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีกิจกรรมอื่น ที่เพิ่มความสัมพันธ์ เช่น มีการนัดทำกิจกรรมร่วมกันออนไลน์]':'Temp88',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [ไม่มีที่กล่าวมาข้างตน]':'Temp89',
'J7 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับว่าการแพร่ระบาดของ COVID-19 มีผลกระทบกับท่านหรือทีมของท่านอย่างไร?':'Temp90',
'K1 มีเรื่องอื่นที่ท่านอยากแชร์เกี่ยวกับการทำงานที่บ้านหรือไม่?':'Temp91',
'Note':'Note'})

In [73]:
covid_df.columns.tolist()

['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'Temp1',
 'Temp2',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 'Te

In [74]:
original_covid_df.columns.tolist()

['Timestamp',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมากที่สุด',
 'D1 ท่า

In [75]:
covid_df.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [76]:
# reorder column

covid_df = covid_df.reindex(columns=['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'Language',
 'AtLeastOneOpenResponse',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS12',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB5',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS5',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'PerformanceOthers',
 'PerB1item',
 'PerS1item',
 'Temp1',
 'Temp2',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 'Temp24',
 'Temp25',
 'Temp26',
 'Temp27',
 'Temp28',
 'Temp29',
 'Temp30',
 'Temp31',
 'Temp32',
 'Temp33',
 'Temp34',
 'Temp35',
 'Temp36',
 'Temp37',
 'Temp38',
 'Temp39',
 'Temp40',
 'Temp41',
 'Temp42',
 'Temp43',
 'Temp44',
 'Temp45',
 'Temp46',
 'Temp47',
 'Temp48',
 'Temp49',
 'Temp50',
 'Temp51',
 'Temp52',
 'Temp53',
 'Temp54',
 'Temp55',
 'Temp56',
 'Temp57',
 'Temp58',
 'Temp59',
 'Temp60',
 'Temp64',
 'Temp65',
 'Temp65.1',
 'Temp66',
 'Temp67',
 'Temp68',
 'Temp69',
 'Temp70',
 'Temp71',
 'Temp72',
 'Temp73',
 'Temp74',
 'Temp75',
 'Temp76',
 'Temp77',
 'Temp78',
 'Temp79',
 'Temp80',
 'Temp81',
 'Temp82',
 'Temp83',
 'Temp84',
 'Temp85',
 'Temp86',
 'Temp87',
 'Temp88',
 'Temp90',
 'Temp91',
 ])

In [77]:
covid_df.shape

(167, 165)

# questionnaire verion 1

In [78]:
covid_df_v1 = pd.read_excel('แบบสอบถามเรื่องผลกระทบจาก COVID-19 กับนักพัฒนาซอฟตแวร์ (Responses)-1st_round.xlsx')
original_covid_df_v1 = covid_df_v1.copy()

In [79]:
covid_df_v1.shape

(88, 179)

In [80]:
covid_df_v1 = covid_df_v1.rename(columns={'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว':'Confirm_REMOVE',
'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน':'Office_work_before_REMOVE',
'A1 ตำแหน่งงานปัจจุบันของท่าน':'RolesIncludeDeveloper',
'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ':'YearsOfExperience',
'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?':'YearsOfWorkFromHomeExperience',
'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ':'Fulltime',
'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?':'Coinhabitants',
'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?':'Children',
'A7 ท่านอายุเท่าไหร่?':'Age',
'A8 โปรดระบุเพศของท่าน':'Gender',
'A9 ท่านจบการศึกษาสูงสุดในระดับใด?':'Education',
'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?':'OrganizationSize',
'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?':'Disabilities',
'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด':'B1_REMOVE',
'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน':'COVIDStatus',
'C2 เลือกข้อที่ตรงกับท่านมากที่สุด':'Isolation',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะติดเชื้อ COVID-19]':'FR1',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR2',
'D1 ท่านรู้สึกกลัวว่า [คนในครอบครัวของท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR3',
'D2 ตอนนี้ท่านรู้สึก [หวาดกลัว]':'FR4',
'D2 ตอนนี้ท่านรู้สึก [ตนเองไม่สามารถทำอะไรเพื่อแก้ปัญหาได้]':'FR5',
'D2 ตอนนี้ท่านรู้สึก [ตกใจกลัวอย่างที่สุด]':'FR6',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านหวาดกลัวว่าท่านจะไม่มีเงินหากท่านไม่สามารถทำงานได้ในช่วง 2-3 เดือนข้างหน้าหรือไม่?]':'FR7',
'D3 โปรดตามคำถามต่อไปนี้ [ปกติแล้วท่านสามารถหายจากอาการเจ็บป่วยได้รวดเร็วหรือไม่?]':'FR8',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านมีเพื่อนที่สามารถติดต่อได้ทางโทรศัพท์ อีเมล หรืออื่นๆ หรือไม่ ในเวลาที่ท่านรู้สึกโดดเดี่ยว]':'FR9',
'How many different prescription medications are you on?':'FR10',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปรึกษากับคนอื่นเกี่ยวกับข้อมูลเรื่อง COVID-19 ที่ข้าพเจ้าได้รับมา]':'DP1',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ามีชุดยังชีพฉุกเฉิน]':'DP2',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าได้วางแผนไว้แล้วว่าต้องปฏิบัติตัวอย่างไรหากข้าพเจ้าตรวจพบเชื้อ COVID-19]':'DP3',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปฏิบัติตามคำแนะนำของหน่วยงานของรัฐเกี่ยวกับการปฏิบัติตัวช่วง COVID-19 อย่างเคร่งครัด]':'DP4',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ารู้จักคนที่จะช่วยข้าพเจ้าได้หากข้าพเจ้าจะต้องกักตัว]':'DP5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [สถาพแวดล้อมของที่ทำงานที่บ้านของข้าพเจ้าปราศจากสิ่งรบกวน]':'Erg1',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ระดับของเสียงรบกวนที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่ยอมรับได้ (ไม่รบกวนการทำงาน)]':'Erg2',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [อุณหภูมิของที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg3',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [แสงในที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg4',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [เก้าอี้ทำงานของข้าพเจ้ามีความสบาย]':'Erg5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ที่ทำงานที่บ้านของข้าพเจ้ามีความเหมาะสมกับการทำงาน]':'Erg6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ข้าพเจ้าสามารถนำอุปกรณ์ (เช่น จอภาพ) จากที่ทำงานกลับมาที่บ้านได้]':'OS1',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้ออุปกรณ์ใหม่ให้พนักงานเพื่อทำงานที่บ้าน]':'OS2',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้อซอฟต์แวร์ที่จำเป็นสำหรับการทำงานที่บ้าน]':'OS3',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะจ่ายค่าอินเตอร์เน็ตให้ข้าพเจ้า]':'OS4',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะไม่ถูกไล่ออก]':'OS5',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะได้รับเงินเดือนเหมือนเดิม]':'OS6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าสามารถลาหยุดหากข้าพเจ้าป่วยหรือลาเพื่อไปดูแลคนในครอบครัวได้]':'OS7',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าองค์กรมีความเข้าใจหากผลการทำงานของข้าพเจ้าแย่ลง]':'OS8',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ายังคงมีการประชุมประจำเช่นเดิม (เช่น ผ่านระบบ video call)]':'OS9',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการประชุม (เช่น ผ่าน video call) น้อยลง]':'OS10',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าให้กำลังใจพนักงาน โดยให้มาพบเจอกัน (แบบออนไลน์) เป็นประจำ]':'OS11',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการสร้างกิจกรรม virtual social events (ผ่านระบบ via video chat) ให้ทุกคนในทีม]':'OS12',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ในแต่ละวัน ข้าพเจ้าทำงานโดยเปิด video call หรือ audio call ค้างไว้กับเพื่อนร่วมงานในทีม]':'OS13',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าส่งอาหารให้กับพนักงานที่บ้าน]':'OS14',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้บุตรของข้าพเจ้าได้ทำระหว่างที่ข้าพเจ้าทำงาน]':'OS15',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้พนักงานออกกำลังกายที่บ้าน]':'OS16',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าสนับสนุนให้พนักงานใช้เวลาช่วงนี้ในการฝึกฝนตนเอง (professional training)]':'OS17',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีระบบที่รองรับการทำงานที่บ้านได้อย่างดี (เช่น source control, VPN, remote desktop, file sharing)]':'OS18',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการตรวจเช็ค commits, change request หรือ pull request (peer code review)]':'OS19',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้าใช้ระบบ build system เพื่อทำการ compile และ test software]':'OS20',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ากำลังปรับปรุงเอกสารและกระบวนการทำงาน (เช่น การ approve change request/pull request)]':'OS21',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [มีคนในองค์กรที่ช่วยจัดลำดับความสำคัญของงานและตรวจเช็คข้อมูลใน backlog ให้ถูกต้องเสมอ]':'OS22',
'F2 มีสิ่งอื่นใดที่บริษัทของท่านกำลังทำหรือควรจะทำเพื่อช่วยท่านระหว่างทำงานที่บ้าน?':'Temp1',
'F3 มีสิ่งใดที่บริษัทของท่านทำแล้วท่านคิดว่าทำให้การทำงานที่บ้านลำบากมากขึ้น?':'Temp2',

'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5B1',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5B2',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5B3',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5B4',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนใจ]':'WHO5B5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB1',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB2',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านเลือกที่จะไม่ทำงานทั้งที่ควรจะต้องทำงานนั้นๆ บ่อยแค่ไหน?]':'HPQB3',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQB4',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQB5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQB6',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQB7',
'G3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQB8',
'G4 ตอนที่ท่านทำงานที่ออฟฟิศ ในบริเวณโต๊ะที่ท่านทำงานมีคนจำนวนกี่คน (รวมตัวท่านเอง)?':'Temp3',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านมีประชุมวันละประมาณกี่ครั้ง?':'Temp4',
#'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',
'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',    #v1
#'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',
'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',    #v1
#'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7',
'G8 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7', #v1
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5S1',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5S2',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5S3',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5S4',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนจ]':'WHO5S5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS1',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS2',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่ทำงานทั้งที่ควรจะต้องทำงานบ่อยแค่ไหน?]':'HPQS3',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQS4',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQS5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQS6',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQS7',
'H3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ตั้งแต่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQS8',
'H4 ตอนที่ท่านทำงานที่บ้าน ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp7.1',
'H5 ท่านทำงานที่บ้านเป็นประจำหรือไม่ก่อนเกิดการแพร่ระบาดของ COVID-19?':'Temp8',
#'H5 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',
'H6 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',   #v1
#'H6 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',
'H7 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',   #v1
'H8 โปรดอธิบายว่าสถานที่ทำงานที่บ้านของท่านเป็นอย่างไร ':'Temp11',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Laptop]':'Temp12',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Desktop]':'Temp13',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Multiple screens]':'Temp14',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Headphones]':'Temp15',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Keyboard]':'Temp16',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Mouse]':'Temp17',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะ]':'Temp18',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Whiteboard]':'Temp19',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะแบบ Ergonomic]':'Temp20',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [เก้าอี้แบบ Ergonomic]':'Temp21',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [แสงสว่างตามธรรมชาติ]':'Temp22',
#'H7 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11',
'H10 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11.1',   #v1
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านดีกว่าที่ทำงาน]':'Temp24',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ดี]':'Temp25',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [บริหารจัดการงานได้ดีขึ้น (เช่น การจัดลำดับความสำคัญของงาน การเลือกงาน)]':'Temp26',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาที่โฟกัสกับงานมากขึ้น]':'Temp27',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีการประชุมที่มีประสิทธิภาพขึ้น]':'Temp28',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีส่ิงรบกวนน้อยลง]':'Temp29',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใช้เวลาเดินทางน้อยลง]':'Temp30',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานที่ยืดหยุ่นขึ้น]':'Temp31',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีกิจกรรมออกกำลังกายมากขึ้น]':'Temp32',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้หยุดพักระหว่างทำงานบ่อยขึ้น]':'Temp33',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีปัญหาสุขภาพน้อยลง]':'Temp34',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใด้อยู่ใกล้ชิดกับครอบครัวมากขึ้น]':'Temp35',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีค่าใช้จ่ายน้อยลงในการเดินทาง อาหาร อื่นๆ]':'Temp36',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้สวมใส่เสื้อผ้าที่สบายมากขึ้น]':'Temp37',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานให้เสร็จมากขึ้น]':'Temp38',
'H12 มีประโยชน์อื่นๆ ที่ท่านได้รับจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp39',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านแย่กว่าที่ทำงาน]':'Temp40',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีอุปกรณ์ hardware, monitor หรืออุปกรณ์ที่จำเป็นไม่เพียงพอ]':'Temp41',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ไม่ดี]':'Temp42',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีส่ิงรบกวนมากขึ้น]':'Temp43',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีปัญหาการเชื่อมต่อ (เช่น Internet, VPN, Remote desktop)]':'Temp44',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [รับรู้ว่าเพื่อนร่วมงานกำลังทำอะไรอยู่น้อยลง]':'Temp45',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สื่อสารกับเพื่อนร่วมงานยาก]':'Temp46',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความยากลำบากในการใช้งานเครื่องมือที่ช่วยทำงานร่วมกัน (collaboration tools)]':'Temp47',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่สามารถทำงานได้ เพราะต้องคอยงานจากเพื่อนร่วมงาน]':'Temp48',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีประชุมเยอะเกินไป]':'Temp49',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีกิจกรรมออกกำลังกายน้อยลง]':'Temp50',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดการปฏิสัมพันธ์กับเพื่อนร่วมงาน]':'Temp51',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ได้หยุดพักระหว่างทำงานน้อยลง]':'Temp52',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ทำกิจกรรมประจำได้น้อยลง]':'Temp53',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดเส้นแบ่งระหว่างงานและชีวิตส่วนตัว]':'Temp54',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีเวลาในการทำงานให้เสร็จน้อยลง]':'Temp55',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีคนดูแลบุตร]':'Temp56',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีกำลังใจในการทำงาน]':'Temp57',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความลำบากในการหาอาหาร]':'Temp58',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ที่ทำงานที่บ้านมี ergonomic ต่ำ]':'Temp59',
'H14 มีปัญหาหรือความลำบากอื่นๆ ที่ท่านได้ประสบจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp60',
'H15 เครื่องมือซอฟต์แวร์ที่ช่วยในการทำงาน (software tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp61',
'H16 เครื่องมือฮาร์ดแวร์ที่ช่วยในการทำงาน (hardware tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp62',
'H17 กระบวนการทำงาน (process) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp63',
'H18 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อดีกับท่านอย่างไร?':'Temp64',
'H19 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อเสียกับท่านอย่างไร?':'Temp65',
'I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?':'PerformanceOthers',
'I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?':'PerB1item',
'I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?':'PerS1item',
'I4 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับประสิทธิผลในการทำงานหรือไม่':'Temp65.1',
'J1 เปรียบเทียบกับการทำงานในออฟฟิศ ประสิทธิผล (productivity) ของทีมท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp66',
'J2 วัฒนธรรมการทำงานของทีมของท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp67',
'J3 ทีมของท่านช่วยเหลือท่านอย่างไรบ้างในช่วงการแพร่ระบาดของ COVID-19?':'Temp68',
'J4 เมื่อเปรียบเทียบกับการทำงานในออฟฟิศ ท่านคิดว่าความสามารถของทีมในการทำงานเพื่อให้เสร็จสิ้นให้ทันเดดไลน์เปลี่ยนแปลงไปหรือไม่? หากใช่ โปรดอธิบาย':'Temp69',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการได้รับข้อมูลใดๆ จากเพื่อนร่วมทีม]':'Temp70',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การรับรู้ว่าเพื่อนร่วมทีมกำลังทำงานอะไรอยู่]':'Temp71',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ประสิทธิภาพของการสื่อสารกับเพื่อนร่วมทีม]':'Temp72',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความง่ายในการสื่อสารกับเพื่อนร่วมทีม]':'Temp73',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [งานที่ข้าพเจ้าทำให้กับทีม]':'Temp74',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับเพื่อนร่วมทีม]':'Temp75',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับหัวหน้างาน]':'Temp76',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบประจำ]':'Temp77',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบไม่ได้นัดมาก่อน]':'Temp78',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [คุณภาพของการประชุมแบบประจำ]':'Temp79',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความรู้สึกของการได้มีปฏิสัมพันธ์ทางสังคมกับทีม]':'Temp80',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การแบ่งปันความรู้ในทีม]':'Temp81',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการทานอาหาร หรือ coffee break ร่วมกันแบบ virtual]':'Temp82',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการพบปะเพื่อพูดคุยกันแบบ 1-1]':'Temp83',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [แบ่งปันภาพถ่ายขำขันหรือเรื่องตลก]':'Temp84',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เล่นเกมส์ออนไลน์ด้วยกัน]':'Temp85',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการตั้ง online meeting channel เพื่อกิจกรรมสนุกสนาน]':'Temp86',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เริ่มการประชุมหรือจบการประชุมด้วยเรื่องที่ไม่เกี่ยวกับงาน]':'Temp87',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีกิจกรรมอื่น ที่เพิ่มความสัมพันธ์ เช่น มีการนัดทำกิจกรรมร่วมกันออนไลน์]':'Temp88',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [ไม่มีที่กล่าวมาข้างตน]':'Temp89',
'J7 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับว่าการแพร่ระบาดของ COVID-19 มีผลกระทบกับท่านหรือทีมของท่านอย่างไร?':'Temp90',
'K1 มีเรื่องอื่นที่ท่านอยากแชร์เกี่ยวกับการทำงานที่บ้านหรือไม่?':'Temp91',
'Note':'Note'})

In [81]:
covid_df_v1.columns.tolist()

['Timestamp',
 'Email Address',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'Temp1',
 'Temp2',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'Temp3',
 'Temp4',
 'Temp5',
 'Temp6',
 'Temp7',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'Temp7.1',
 'Temp8'

In [82]:
original_covid_df_v1.columns.tolist()

['Timestamp',
 'Email Address',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมา

In [83]:
covid_df_v1.columns[covid_df_v1.columns.duplicated()]

Index([], dtype='object')

In [84]:
# reorder column

covid_df_v1 = covid_df_v1.reindex(columns=['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'Language',
 'AtLeastOneOpenResponse',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS12',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB5',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS5',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'PerformanceOthers',
 'PerB1item',
 'PerS1item',
 'Temp1',
 'Temp2',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 'Temp24',
 'Temp25',
 'Temp26',
 'Temp27',
 'Temp28',
 'Temp29',
 'Temp30',
 'Temp31',
 'Temp32',
 'Temp33',
 'Temp34',
 'Temp35',
 'Temp36',
 'Temp37',
 'Temp38',
 'Temp39',
 'Temp40',
 'Temp41',
 'Temp42',
 'Temp43',
 'Temp44',
 'Temp45',
 'Temp46',
 'Temp47',
 'Temp48',
 'Temp49',
 'Temp50',
 'Temp51',
 'Temp52',
 'Temp53',
 'Temp54',
 'Temp55',
 'Temp56',
 'Temp57',
 'Temp58',
 'Temp59',
 'Temp60',
 'Temp64',
 'Temp65',
 'Temp65.1',
 'Temp66',
 'Temp67',
 'Temp68',
 'Temp69',
 'Temp70',
 'Temp71',
 'Temp72',
 'Temp73',
 'Temp74',
 'Temp75',
 'Temp76',
 'Temp77',
 'Temp78',
 'Temp79',
 'Temp80',
 'Temp81',
 'Temp82',
 'Temp83',
 'Temp84',
 'Temp85',
 'Temp86',
 'Temp87',
 'Temp88',
 'Temp90',
 'Temp91',
 ])

# merge covid_df V1 and V2

In [85]:
print('before')
print(covid_df.shape)
print(covid_df_v1.shape)

#drop those Temp column
# https://www.skytowner.com/explore/dropping_columns_whose_label_contains_a_substring_in_pandas

covid_df = covid_df.loc[:,~covid_df.columns.str.contains('Temp')]
covid_df_v1 = covid_df_v1.loc[:,~covid_df_v1.columns.str.contains('Temp')]

print('after')
print(covid_df.shape)
print(covid_df_v1.shape)

print('merge')
covid_df = pd.concat([covid_df,covid_df_v1])
print(covid_df.shape)

before
(167, 165)
(88, 165)
after
(167, 90)
(88, 90)
merge
(255, 90)


# start preprocessing

In [86]:
# add id column
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas
covid_df.insert(0, 'ID', range(1, 1 + len(covid_df)))

In [87]:
# remove unnamed column
# del covid_df['Unnamed: 0']

In [88]:
# move Timestamp column to the beginning
covid_df.insert(0, 'Timestamp', covid_df.pop('Timestamp'))

In [89]:
# remove Email column
# del covid_df['Email Address']

In [90]:
# remove Confirm_REMOVE column
del covid_df['Confirm_REMOVE']
del covid_df['B1_REMOVE']

In [91]:
# filter Office_work_before_REMOVE with no answer
covid_df.shape
covid_df = covid_df[covid_df['Office_work_before_REMOVE'] == 'ใช่']
del covid_df['Office_work_before_REMOVE']
covid_df.shape

(223, 88)

In [92]:
# fill Language with 'Thai'
covid_df['Language'] = 'Thai'

In [93]:
#covid_df
covid_df.to_csv('covid_data.csv', index=False)

## Years of experience

In [94]:
covid_df.YearsOfExperience.value_counts()

1-5 ปี           77
มากกว่า 10 ปี    71
6-10 ปี          68
น้อยกว่า 1 ปี     7
Name: YearsOfExperience, dtype: int64

In [95]:
# df["column1"].replace({"a": "x", "b": "y"}, inplace=True)

covid_df["YearsOfExperience"].replace({
    "น้อยกว่า 1 ปี": 0.5,
    "1-5 ปี": 2,
    "6-10 ปี": 8,
    "มากกว่า 10 ปี" : 15,
}, inplace=True)

covid_df.YearsOfExperience.value_counts()

2.0     77
15.0    71
8.0     68
0.5      7
Name: YearsOfExperience, dtype: int64

## AtLeastOneOpenResponse


In [96]:
covid_df["AtLeastOneOpenResponse"] = covid_df["AtLeastOneOpenResponse"].fillna(0)

## YearsOfWorkFromHomeExperience

In [97]:
covid_df["YearsOfWorkFromHomeExperience"].replace({
    "น้อยกว่า 1 ปี": 0.5,
    "1-5 ปี": 2,
    "6-10 ปี": 8,
    "มากกว่า 10 ปี" : 15,
}, inplace=True)

covid_df.YearsOfExperience.value_counts()

2.0     77
15.0    71
8.0     68
0.5      7
Name: YearsOfExperience, dtype: int64

## Fulltime

In [98]:
covid_df.Fulltime.value_counts()

Full time    219
Part time      4
Name: Fulltime, dtype: int64

In [99]:
covid_df["Fulltime"].replace({
    "Full time": 1,
    "Part time": 0,
}, inplace=True)

covid_df.Fulltime.value_counts()

1    219
0      4
Name: Fulltime, dtype: int64

## Coinhabitants

In [100]:
covid_df.Coinhabitants.value_counts()

4.0     51
3.0     49
2.0     41
5.0     33
1.0     21
6.0     13
7.0      4
8.0      3
4 คน     2
2 คน     1
9.0      1
1-4      1
ุ6       1
1 คน     1
3 คน     1
Name: Coinhabitants, dtype: int64

In [101]:
covid_df["Coinhabitants"].replace({
    "4 คน": 4,
    "2 คน": 2,
    "1-4":4,
    "ุ6": 6,
    "1 คน": 1,
    "3 คน": 3
}, inplace=True)

covid_df.Coinhabitants.value_counts()

4.0    54
3.0    50
2.0    42
5.0    33
1.0    22
6.0    14
7.0     4
8.0     3
9.0     1
Name: Coinhabitants, dtype: int64

In [102]:
print("Column: Children")
covid_df.Children.value_counts()

covid_df["Children"].replace({
    "ไม่มี": 0,
    "-": 0,
    "0 คน": 0,
}, inplace=True)

print(covid_df.Children.value_counts())
print(covid_df.Children.isna().sum())

Column: Children
0.0    179
1.0     26
2.0     14
3.0      2
4.0      1
Name: Children, dtype: int64
1


In [103]:
print("Column: Age")
print(covid_df.Age.value_counts())

covid_df["Age"].replace({
    "20-24": 2,
    "25-29": 3,
    "30-34": 4,
    "35-39": 5,
    "40-44": 6,
    "45-49": 7,
    "50-54": 8,
    "55-59": 9,
}, inplace=True)
print(covid_df.Age.value_counts())

Column: Age
30-34    78
25-29    64
35-39    33
20-24    22
45-49    12
40-44    10
55-59     2
50-54     2
Name: Age, dtype: int64
4    78
3    64
5    33
2    22
7    12
6    10
9     2
8     2
Name: Age, dtype: int64


In [104]:
print("Column: Gender")
print(covid_df.Gender.value_counts())

covid_df["Gender"].replace({
    "เพศชาย": "Male",
    "เพศหญิง": "Female",
    "เพศที่สาม/ไม่ต้องการะบุ": "Not identified",
}, inplace=True)

print(covid_df.Gender.value_counts())

Column: Gender
เพศชาย                     130
เพศหญิง                     86
เพศที่สาม/ไม่ต้องการะบุ      6
Name: Gender, dtype: int64
Male              130
Female             86
Not identified      6
Name: Gender, dtype: int64


In [105]:
print("Column: Education")
print(covid_df.Education.value_counts())
print("Null:", covid_df["Education"].isna().sum())

covid_df["Education"].replace({
    "ปริญญาตรีหรือเท่าเทียม": "2",
    "ปริญญาโทหรือเท่าเทียม": "3",
    "ปริญญาเอกหรือเท่าเทียม": "4",
}, inplace=True)

print(covid_df.Education.value_counts())

Column: Education
ปริญญาตรีหรือเท่าเทียม    146
ปริญญาโทหรือเท่าเทียม      72
ปริญญาเอกหรือเท่าเทียม      5
Name: Education, dtype: int64
Null: 0
2    146
3     72
4      5
Name: Education, dtype: int64


In [106]:
print("Column: OrganizationSize")
print(covid_df.OrganizationSize.value_counts())
print("Null:", covid_df["OrganizationSize"].isna().sum())

covid_df["OrganizationSize"].replace({
    "0 ถึง 9": 1,
    "10 ถึง 99": 2,
    "100 ถึง 999": 3,
    "1,000 ถึง 9,999": 4,
    "10,000 ถึง 99,999": 5,
    "100,000 หรือมากกว่า": 6
}, inplace=True)

print(covid_df.OrganizationSize.value_counts())

Column: OrganizationSize
100 ถึง 999            97
1,000 ถึง 9,999        51
10 ถึง 99              50
10,000 ถึง 99,999      13
0 ถึง 9                 6
100,000 หรือมากกว่า     6
Name: OrganizationSize, dtype: int64
Null: 0
3    97
4    51
2    50
5    13
1     6
6     6
Name: OrganizationSize, dtype: int64


In [107]:
print("Column: Disabilities")
print(covid_df.Disabilities.value_counts())
print("Null:", covid_df["Disabilities"].isna().sum())

covid_df["Disabilities"].replace({
    "มี": 2,
    "ไม่มี": 0,
    "อาจจะ/ไม่แน่ใจ": 1,
}, inplace=True)

print(covid_df.Disabilities.value_counts())

Column: Disabilities
ไม่มี             214
อาจจะ/ไม่แน่ใจ      5
มี                  4
Name: Disabilities, dtype: int64
Null: 0
0    214
1      5
2      4
Name: Disabilities, dtype: int64


In [ ]:
print("Column: COVIDStatus")
print(covid_df.COVIDStatus.value_counts())
print("Null:", covid_df["COVIDStatus"].isna().sum())

# https://discuss.dizzycoding.com/converting-pandas-column-of-comma-separated-strings-into-dummy-variables/

# covidstatus = covid_df['COVIDStatus'].str.get_dummies(sep=',')   อันนี้ไม่เวิค 

# https://www.datasciencemadesimple.com/replace-a-substring-of-a-column-in-pandas-python/
# df1.replace(regex=['zona'], value='Arizona'

covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['ไม่มีข้อที่ตรงกับข้าพเจ้า'], value='0')
covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['ตอนนี้ข้าพเจ้ากำลังกักตัว (quarantine) หรือเพิ่งกักตัวไปไม่นาน (โดยไม่เดินทางออกจากบ้านหรือที่พัก)'], value='1')
covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['เพื่อนสนิทของข้าพเจ้า เคยได้รับการตรวจและติดเชื้อ COVID-19'], value='2')
covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['สมาชิกในครอบครัวของข้าพเจ้า เคยได้รับการตรวจและติดเชื้อ COVID-19'], value='3')
covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['สมาชิกในบ้านของข้าพเจ้า (แต่ไม่ใช่ญาติ) เคยได้รับการตรวจและติดเชื้อ COVID-19'], value='4')
covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['ข้าพเจ้าเคยได้รับการตรวจและติดเชื้อ COVID-19'], value='5')

covid_df["COVIDStatus"] = covid_df["COVIDStatus"].replace(regex=['ตอนนี้ข้าพเจ้ากำลังกักตัว (quarantine) หรือเพิ่งกักตัวไปไม่นาน (โดยไม่เดินทางออกจากบ้านหรือที่พัก)  '], value='1')

covid_df["COVIDStatus"] = covid_df["COVIDStatus"].str.replace('สมาชิกในบ้านของข้าพเจ้า (แต่ไม่ใช่ญาติ) เคยได้รับการตรวจและติดเชื้อ COVID-19', '4')


print(covid_df.COVIDStatus.value_counts())                                                                                                                                                             

# covid_df["OrganizationSize"].replace({
#     "0 ถึง 9": 1,
#     "10 ถึง 99": 2,
#     "100 ถึง 999": 3,
#     "1,000 ถึง 9,999": 4,
#     "10,000 ถึง 99,999": 5,
#     "100,000 หรือมากกว่า": 6
# }, inplace=True)

# print(covid_df.OrganizationSize.value_counts())



In [109]:
covid_df.to_csv('covid_data.csv', index=False, encoding='utf-8-sig')

In [115]:
covid_df_2 = pd.read_csv('covid_data.csv', encoding='utf-8-sig')
covid_df_2.head()
print(covid_df_2.COVIDStatus.value_counts())
covid_df_2["COVIDStatus"] = covid_df_2["COVIDStatus"].replace(regex=[' '], value='') 
print(covid_df_2.COVIDStatus.value_counts())

covidstatus = covid_df_2['COVIDStatus'].str.get_dummies(sep=',').rename(lambda x: 'COVIDStatus_' + x, axis='columns')
covidstatus



covid_df_2 = pd.concat([covid_df_2, covidstatus], axis=1)

0             124
2              27
3              17
3, 2            9
5               7
2, 0            6
5, 3            4
1               4
4               3
5, 2            3
5, 4, 3, 2      3
5, 3, 2         2
3, 2, 1         2
5, 2, 1         2
4, 2            2
3, 1            1
4, 3, 2         1
4, 2, 1         1
4, 3            1
4, 1            1
2, 1            1
5, 1            1
5, 0            1
Name: COVIDStatus, dtype: int64
0          124
2           27
3           17
3,2          9
5            7
2,0          6
5,3          4
1            4
4            3
5,2          3
5,4,3,2      3
5,3,2        2
3,2,1        2
5,2,1        2
4,2          2
3,1          1
4,3,2        1
4,2,1        1
4,3          1
4,1          1
2,1          1
5,1          1
5,0          1
Name: COVIDStatus, dtype: int64


In [116]:
covid_df_2.head()

,Timestamp,ID,Language,AtLeastOneOpenResponse,RolesIncludeDeveloper,YearsOfExperience,YearsOfWorkFromHomeExperience,Fulltime,Coinhabitants,Children,Age,Gender,Education,OrganizationSize,Disabilities,COVIDStatus,Isolation,FR1,FR2,FR3,FR4,FR5,FR6,FR7,FR8,...,HPQB6,HPQB7,HPQB8,WHO5S1,WHO5S2,WHO5S3,WHO5S4,WHO5S5,HPQS1,HPQS2,HPQS3,HPQS4,HPQS5,HPQS6,HPQS7,HPQS8,PerformanceOthers,PerB1item,PerS1item,COVIDStatus_0,COVIDStatus_1,COVIDStatus_2,COVIDStatus_3,COVIDStatus_4,COVIDStatus_5
0,19:26.1,1,Thai,0,Software Developer,8.0,2.0,1,2,0.0,4,Male,3,5,0,0,ข้าพเจ้าออกจากบ้านสม่ำเสมอเพื่อไปทำงานหรือผ่อน...,ไม่ใช่,ไม่ใช่,อาจจะ/ไม่แน่ใจ,ไม่ใช่,ไม่ใช่,NaN,ไม่ใช่,ใช่,...,บางครั้ง,ไม่เคย,ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้าย...,มากกว่าครึ่ง,ตลอดเวลา,มากกว่าครึ่ง,มากกว่าครึ่ง,มากกว่าครึ่ง,บ่อย,ไม่บ่อย,บ่อย,ไม่บ่อย,NaN,บางครั้ง,ไม่ทราบ/ไม่แน่ใจ,ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้าย...,8.0,8.0,8.0,1,0,0,0,0,0
1,30:19.1,2,Thai,0,Software Developer,2.0,8.0,1,4,0.0,4,Male,2,2,0,0,ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท...,อาจจะ/ไม่แน่ใจ,ไม่ใช่,ไม่ใช่,อาจจะ/ไม่แน่ใจ,อาจจะ/ไม่แน่ใจ,NaN,ใช่,อาจจะ/ไม่แน่ใจ,...,บ่อย,ไม่บ่อย,ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนให...,น้อยกว่าครึ่ง,บางครั้ง,น้อยกว่าครึ่ง,บางครั้ง,บางครั้ง,บางครั้ง,บางครั้ง,บางครั้ง,บางครั้ง,NaN,บางครั้ง,บางครั้ง,ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้...,5.0,4.0,5.0,1,0,0,0,0,0
2,30:34.2,3,Thai,0,Software Developer,15.0,2.0,1,5,0.0,4,Male,2,4,0,0,ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท...,อาจจะ/ไม่แน่ใจ,ไม่ใช่,ไม่ใช่,ไม่ใช่,ไม่ใช่,NaN,ไม่ใช่,อาจจะ/ไม่แน่ใจ,...,ไม่ทราบ/ไม่แน่ใจ,ไม่บ่อย,ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนให...,มากกว่าครึ่ง,มากกว่าครึ่ง,มากกว่าครึ่ง,มากกว่าครึ่ง,เป็นส่วนใหญ่,บางครั้ง,บางครั้ง,ไม่ทราบ/ไม่แน่ใจ,ไม่ทราบ/ไม่แน่ใจ,NaN,ไม่ทราบ/ไม่แน่ใจ,ไม่ทราบ/ไม่แน่ใจ,ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้าย...,9.0,8.0,8.0,1,0,0,0,0,0
3,13:25.4,4,Thai,0,Machine Learning Engineer,8.0,2.0,1,1,0.0,3,Male,2,4,0,0,ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท...,ไม่ใช่,ไม่ใช่,ไม่ใช่,อาจจะ/ไม่แน่ใจ,ใช่,NaN,ใช่,อาจจะ/ไม่แน่ใจ,...,บางครั้ง,บางครั้ง,ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนให...,เป็นส่วนใหญ่,ตลอดเวลา,มากกว่าครึ่ง,เป็นส่วนใหญ่,มากกว่าครึ่ง,บางครั้ง,บางครั้ง,ไม่บ่อย,บางครั้ง,NaN,บางครั้ง,บ่อย,ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนให...,8.0,8.0,8.0,1,0,0,0,0,0
4,23:00.7,5,Thai,0,Software Developer,8.0,2.0,1,4,0.0,3,Not identified,2,3,0,2,ข้าพเจ้าออกจากบ้านสม่ำเสมอเพื่อไปทำงานหรือผ่อน...,อาจจะ/ไม่แน่ใจ,อาจจะ/ไม่แน่ใจ,ใช่,ไม่ใช่,ไม่ใช่,NaN,ใช่,ใช่,...,บ่อย,บ่อย,ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนให...,เป็นส่วนใหญ่,เป็นส่วนใหญ่,มากกว่าครึ่ง,เป็นส่วนใหญ่,บางครั้ง,บ่อย,ไม่บ่อย,บางครั้ง,บางครั้ง,NaN,ไม่บ่อย,ไม่บ่อย,ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้าย...,7.0,8.0,8.0,0,0,1,0,0,0


In [121]:
print("columns: Isolation")
print(covid_df_2.Isolation.value_counts())
print("Null:", covid_df_2["Isolation"].isna().sum())

covid_df_2["Isolation"].replace({
    "ข้าพเจ้าออกจากบ้านสม่าเสมอ และข้าพเจ้าไม่ได้รักษาระยะห่างกับผู้อื่น": 1,
    "ข้าพเจ้าออกจากบ้านสม่ำเสมอเพื่อไปทำงานหรือผ่อนคลาย แต่ข้าพเจ้ารักษาระยะห่างกับผู้อื่น": 2,
    "ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท่านั้น (เช่น ซื้ออาหาร พบแพทย์/รับการรักษา ออกกำลังกาย)": 3,
    "ข้าพเจ้าจะไม่ออกจากบ้าน ไม่ว่าจะด้วยเหตุผลใดก็ตาม" : 4
}, inplace=True)

print(covid_df_2.Isolation.value_counts())

columns: Isolation
ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท่านั้น (เช่น ซื้ออาหาร พบแพทย์/รับการรักษา ออกกำลังกาย)    145
ข้าพเจ้าออกจากบ้านสม่ำเสมอเพื่อไปทำงานหรือผ่อนคลาย แต่ข้าพเจ้ารักษาระยะห่างกับผู้อื่น                     70
ข้าพเจ้าออกจากบ้านสม่าเสมอ และข้าพเจ้าไม่ได้รักษาระยะห่างกับผู้อื่น                                        4
ข้าพเจ้าจะไม่ออกจากบ้าน ไม่ว่าจะด้วยเหตุผลใดก็ตาม                                                          4
Name: Isolation, dtype: int64
Null: 0
3    145
2     70
1      4
4      4
Name: Isolation, dtype: int64


In [124]:
print("column: FR1-9")
for i in range(1, 10):
    print(covid_df_2['FR' + str(i)].value_counts())
    covid_df_2['FR' + str(i)].replace({
        "ใช่" : 1,
        "ไม่ใช่" : -1,
        "อาจจะ/ไม่แน่ใจ": 0
    }, inplace=True)
    print(covid_df_2['FR' + str(i)].value_counts())

# 1	Yes
# 0	Maybe/Unsure
# -1	No


column: FR1-9
อาจจะ/ไม่แน่ใจ    81
ใช่               79
ไม่ใช่            63
Name: FR1, dtype: int64
 0    81
 1    79
-1    63
Name: FR1, dtype: int64
ไม่ใช่            138
ใช่                47
อาจจะ/ไม่แน่ใจ     38
Name: FR2, dtype: int64
-1    138
 1     47
 0     38
Name: FR2, dtype: int64
ใช่               94
ไม่ใช่            73
อาจจะ/ไม่แน่ใจ    56
Name: FR3, dtype: int64
 1    94
-1    73
 0    56
Name: FR3, dtype: int64
ไม่ใช่            130
อาจจะ/ไม่แน่ใจ     68
ใช่                25
Name: FR4, dtype: int64
-1    130
 0     68
 1     25
Name: FR4, dtype: int64
ไม่ใช่            114
อาจจะ/ไม่แน่ใจ     55
ใช่                54
Name: FR5, dtype: int64
-1    114
 0     55
 1     54
Name: FR5, dtype: int64
ไม่ใช่    28
Name: FR6, dtype: int64
-1.0    28
Name: FR6, dtype: int64
ไม่ใช่            110
ใช่                74
อาจจะ/ไม่แน่ใจ     39
Name: FR7, dtype: int64
-1    110
 1     74
 0     39
Name: FR7, dtype: int64
ใช่               140
อาจจะ/ไม่แน่ใจ     69
ไม่ใช่            

In [126]:
print("column: DP1-5")
for i in range(1, 6):
    print(covid_df_2['DP' + str(i)].value_counts())
    covid_df_2['DP' + str(i)].replace({
        "เห็นด้วยอย่างยิ่ง" : 4,
        "เห็นด้วยพอสมควร" : 2,
        "เห็นด้วยเล็กน้อย": 1,
        "ไม่เห็นด้วย": 0,
        "ไม่รู้/ไม่มีความเห็น": -99
    }, inplace=True)
    print(covid_df_2['DP' + str(i)].value_counts())

# 4	Agree completely
# 3	Agree quite a bit
# 2	Agree somewhat
# 1	Agree a little bit
# 0	Do not agree at all
# -99	Don't know / no opinion
# Note: lv ไม่เท่ากัน
# ของ raph เป็น 1-5 คะแนน นี้มันอะไร!

column: DP1-5
เห็นด้วยพอสมควร         102
เห็นด้วยอย่างยิ่ง        77
เห็นด้วยเล็กน้อย         33
ไม่เห็นด้วย               9
ไม่รู้/ไม่มีความเห็น      2
Name: DP1, dtype: int64
 2     102
 4      77
 1      33
 0       9
-99      2
Name: DP1, dtype: int64
เห็นด้วยพอสมควร         65
ไม่เห็นด้วย             57
เห็นด้วยเล็กน้อย        56
เห็นด้วยอย่างยิ่ง       32
ไม่รู้/ไม่มีความเห็น    13
Name: DP2, dtype: int64
 2     65
 0     57
 1     56
 4     32
-99    13
Name: DP2, dtype: int64
เห็นด้วยพอสมควร         81
เห็นด้วยเล็กน้อย        61
เห็นด้วยอย่างยิ่ง       55
ไม่เห็นด้วย             21
ไม่รู้/ไม่มีความเห็น     4
Name: DP3, dtype: int64
 2.0     81
 1.0     61
 4.0     55
 0.0     21
-99.0     4
Name: DP3, dtype: int64
เห็นด้วยพอสมควร         113
เห็นด้วยอย่างยิ่ง        65
เห็นด้วยเล็กน้อย         38
ไม่เห็นด้วย               6
ไม่รู้/ไม่มีความเห็น      1
Name: DP4, dtype: int64
 2     113
 4      65
 1      38
 0       6
-99      1
Name: DP4, dtype: int64
เห็นด้วยอย่างยิ่ง       

In [128]:
print("column: Erg1-6")
for i in range(1, 7):
    print(covid_df_2['Erg' + str(i)].value_counts())
    covid_df_2['Erg' + str(i)].replace({
        "เห็นด้วยอย่างยิ่ง" : 3,
        "เห็นด้วย" : 2,
        "เห็นด้วยพอสมควร": 1,
        "ค่อนข้างไม่เห็นด้วย": -1,
        "ไม่เห็นด้วย": -2,
        "ไม่เห็นด้วยอย่างยิ่ง": -3    }, inplace=True)
    print(covid_df_2['Erg' + str(i)].value_counts())

# 3	Strongly Agree
# 2	Agree
# 1	Somewhat Agree
# -1	Somewhat Disagree
# -2	Disagree
# -3	Strongly Disagree


column: Erg1-6
เห็นด้วยพอสมควร         64
เห็นด้วย                60
เห็นด้วยอย่างยิ่ง       46
ค่อนข้างไม่เห็นด้วย     26
ไม่เห็นด้วย             17
ไม่เห็นด้วยอย่างยิ่ง    10
Name: Erg1, dtype: int64
 1    64
 2    60
 3    46
-1    26
-2    17
-3    10
Name: Erg1, dtype: int64
เห็นด้วย                92
เห็นด้วยอย่างยิ่ง       58
เห็นด้วยพอสมควร         47
ค่อนข้างไม่เห็นด้วย     16
ไม่เห็นด้วยอย่างยิ่ง     6
ไม่เห็นด้วย              4
Name: Erg2, dtype: int64
 2    92
 3    58
 1    47
-1    16
-3     6
-2     4
Name: Erg2, dtype: int64
เห็นด้วย                69
เห็นด้วยอย่างยิ่ง       50
เห็นด้วยพอสมควร         48
ค่อนข้างไม่เห็นด้วย     37
ไม่เห็นด้วย             11
ไม่เห็นด้วยอย่างยิ่ง     8
Name: Erg3, dtype: int64
 2    69
 3    50
 1    48
-1    37
-2    11
-3     8
Name: Erg3, dtype: int64
เห็นด้วย                91
เห็นด้วยอย่างยิ่ง       64
เห็นด้วยพอสมควร         44
ค่อนข้างไม่เห็นด้วย     16
ไม่เห็นด้วยอย่างยิ่ง     6
ไม่เห็นด้วย              2
Name: Erg4, dtype: int64


In [139]:
print("column: WHO5B1-5")
for i in range(1, 6):
    print(covid_df_2['WHO5B' + str(i)].value_counts())
    covid_df_2['WHO5B' + str(i)].replace({
        "ตลอดเวลา" : 5,
        "เป็นส่วนใหญ่" : 4,
        "มากกว่าครึ่ง": 3,
        "น้อยกว่าครึ่ง": 2,
        "บางครั้ง": 1,
        "ไม่เคย": 0    
        }, inplace=True)
    print(covid_df_2['WHO5B' + str(i)].value_counts())

print("column: WHO5S1-5")
for i in range(1, 6):
    print(covid_df_2['WHO5S' + str(i)].value_counts())
    covid_df_2['WHO5S' + str(i)].replace({
        "ตลอดเวลา" : 5,
        "เป็นส่วนใหญ่" : 4,
        "มากกว่าครึ่ง": 3,
        "น้อยกว่าครึ่ง": 2,
        "บางครั้ง": 1,
        "ไม่เคย": 0    
        }, inplace=True)
    print(covid_df_2['WHO5S' + str(i)].value_counts())

# 5	All of the time
# 4	Most of the time
# 3	More than half of the time
# 2	Less than half of the time
# 1	Some of the time
# 0	At no time


column: WHO5B1-5
4.0    105
3.0     58
5.0     23
2.0     17
1.0     11
0.0      6
Name: WHO5B1, dtype: int64
4.0    105
3.0     58
5.0     23
2.0     17
1.0     11
0.0      6
Name: WHO5B1, dtype: int64
4.0    89
3.0    66
2.0    24
5.0    22
1.0    12
0.0     9
Name: WHO5B2, dtype: int64
4.0    89
3.0    66
2.0    24
5.0    22
1.0    12
0.0     9
Name: WHO5B2, dtype: int64
4.0    100
3.0     55
5.0     29
2.0     19
1.0     13
0.0      6
Name: WHO5B3, dtype: int64
4.0    100
3.0     55
5.0     29
2.0     19
1.0     13
0.0      6
Name: WHO5B3, dtype: int64
4.0    64
3.0    64
2.0    38
5.0    24
1.0    22
0.0    10
Name: WHO5B4, dtype: int64
4.0    64
3.0    64
2.0    38
5.0    24
1.0    22
0.0    10
Name: WHO5B4, dtype: int64
4.0    75
3.0    66
2.0    31
5.0    31
1.0    15
0.0     4
Name: WHO5B5, dtype: int64
4.0    75
3.0    66
2.0    31
5.0    31
1.0    15
0.0     4
Name: WHO5B5, dtype: int64
column: WHO5S1-5
เป็นส่วนใหญ่     79
มากกว่าครึ่ง     59
น้อยกว่าครึ่ง    37
ตลอดเวลา    

In [144]:
print("column: HPQB1-7")
for i in range(1, 8):
    print(covid_df_2['HPQB' + str(i)].value_counts())
    covid_df_2['HPQB' + str(i)].replace({
        "ตลอดเวลา" : 4,
        "บ่อย" : 3,
        "บางครั้ง": 2,
        "ไม่บ่อย": 1,
        "ไม่ทราบ/ไม่แน่ใจ": 0,  
        }, inplace=True)
    print(covid_df_2['HPQB' + str(i)].value_counts())

print("column: HPQS1-7")
for i in range(1, 8):
    print(covid_df_2['HPQS' + str(i)].value_counts())
    covid_df_2['HPQS' + str(i)].replace({
        "ตลอดเวลา" : 4,
        "บ่อย" : 3,
        "บางครั้ง": 2,
        "ไม่เคย": 1,     # อันนี้ไม่ตรงกับข้างบน
        "ไม่ทราบ/ไม่แน่ใจ": 0,  
        }, inplace=True)
    print(covid_df_2['HPQS' + str(i)].value_counts())

# 4	All of the time
# 3	Most of the time
# 2	Some of the time
# 1	A little of the time
# 0	None of the time

# บ่อย                98
# บางครั้ง            67
# ตลอดเวลา            25
# ไม่ทราบ/ไม่แน่ใจ    19
# ไม่บ่อย             12


column: HPQB1-7
3.0    98
1.0    79
5.0    25
0.0    19
Name: HPQB1, dtype: int64
3.0    98
1.0    79
5.0    25
0.0    19
Name: HPQB1, dtype: int64
1.0    150
0.0     57
3.0     11
5.0      3
Name: HPQB2, dtype: int64
1.0    150
0.0     57
3.0     11
5.0      3
Name: HPQB2, dtype: int64
1.0    137
0.0     64
3.0     16
5.0      3
Name: HPQB3, dtype: int64
1.0    137
0.0     64
3.0     16
5.0      3
Name: HPQB3, dtype: int64
1.0    166
0.0     39
3.0     13
5.0      2
Name: HPQB4, dtype: int64
1.0    166
0.0     39
3.0     13
5.0      2
Name: HPQB4, dtype: int64
1.0    19
0.0     7
3.0     1
Name: HPQB5, dtype: int64
1.0    19
0.0     7
3.0     1
Name: HPQB5, dtype: int64
1.0    153
3.0     46
0.0     19
5.0      2
Name: HPQB6, dtype: int64
1.0    153
3.0     46
0.0     19
5.0      2
Name: HPQB6, dtype: int64
1.0    122
0.0     59
3.0     33
5.0      6
Name: HPQB7, dtype: int64
1.0    122
0.0     59
3.0     33
5.0      6
Name: HPQB7, dtype: int64
column: HPQS1-7
3         85
2         7

In [145]:
print("column: HPQB8")
print(covid_df_2['HPQB8'].value_counts())
covid_df_2['HPQB8'].replace({
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมาก" : 3,
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน" : 2,
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย": 1,
    "ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน" : 0,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย": -1,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน": -2,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมา": -3  
    }, inplace=True)
print(covid_df_2['HPQB8'].value_counts())


print("column: HPQS8")
print(covid_df_2['HPQS8'].value_counts())
covid_df_2['HPQS8'].replace({
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมาก" : 3,
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน" : 2,
    "ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย": 1,
    "ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน" : 0,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย": -1,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน": -2,
    "ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมาก": -3   # อันนี้ข้างบนพิมพ์ผิด ตก 'ก'
    }, inplace=True)
print(covid_df_2['HPQS8'].value_counts())
#     column: HPQB8
# ท่านมีประสิทธิภาพเท่าๆ กับค่าเฉลี่ยของคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน    77
# ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน                   58
# ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย           50
# ท่านมีประสิทธิภาพดีกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมาก                26
# ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านเล็กน้อย          5
# ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน                  2
# ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมา                2
# Name: HPQB8, dtype: int64

column: HPQB8
 0.0    77
 2.0    58
 1.0    50
 3.0    26
-1.0     5
-2.0     2
-3.0     2
Name: HPQB8, dtype: int64
 0.0    77
 2.0    58
 1.0    50
 3.0    26
-1.0     5
-2.0     2
-3.0     2
Name: HPQB8, dtype: int64
column: HPQS8
0                                                                  69
2                                                                  63
1                                                                  45
3                                                                  23
-1                                                                 12
-2                                                                  6
ท่านมีประสิทธิภาพด้อยกว่าคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่านมาก     1
Name: HPQS8, dtype: int64
 0.0    69
 2.0    63
 1.0    45
 3.0    23
-1.0    12
-2.0     6
-3.0     1
Name: HPQS8, dtype: int64


In [152]:
print('OS')

for i in range(1,23):
    covid_df_2['OS'+ str(i)].value_counts()
    covid_df_2["OS"+ str(i)] = covid_df_2["OS"+ str(i)].replace(regex=['ข้อความนี้จริง'], value='y')
    covid_df_2["OS"+ str(i)] = covid_df_2["OS"+ str(i)].replace(regex=['มีประโยชน์หรือน่าจะมีประโยชน์'], value='h')
    covid_df_2["OS"+ str(i)] = covid_df_2["OS"+ str(i)].replace(regex=[' '], value='')

    temp = covid_df_2['OS'+ str(i)].str.get_dummies(sep=',').rename(lambda x: 'OS'+str(i)+'_' + x, axis='columns')
    covid_df_2 = pd.concat([covid_df_2, temp], axis=1)
    del covid_df_2['OS'+ str(i)]



OS
